# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup

import hvplot.pandas
import pandas as pd
import requests

# Import API key

from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame

CityDF = pd.read_csv("output_data/cities.csv")

# Display sample data

CityDF.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,monticello,45.3055,-93.7941,24.97,40,100,3.60,US,1694741675
1,1,colima,19.1667,-104.0000,25.78,95,54,1.00,MX,1694741676
2,2,grytviken,-54.2811,-36.5092,-4.51,97,75,1.47,GS,1694741676
3,3,saint-francois,46.4154,3.9054,12.69,100,0,0.00,FR,1694741676
4,4,lihue,21.9789,-159.3672,28.58,82,75,6.69,US,1694741555


---

### Step 1: Create a map that displays a point for every city in the `CityDF` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure map plot

MapPlot = CityDF.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = .5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500
)

MapPlot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `CityDF` DataFrame to find your ideal weather condition

In [17]:
# Narrow down cities that fit criteria and drop any results with null values

IdealWeatherDF = CityDF[(CityDF["Max Temp"] <= 35) &
                        (CityDF["Max Temp"] >= 17) &
                        (CityDF["Wind Speed"] <= 5) &
                        (CityDF["Cloudiness"] == 0)
                        ]

# Drop any rows with missing value

IdealWeatherDF = IdealWeatherDF.dropna()

# Display data

IdealWeatherDF.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
11,11,mengyin,35.7069,117.9264,24.53,47,0,0.47,CN,1694741681
24,24,quincy,42.2529,-71.0023,18.95,78,0,2.06,US,1694741535
29,29,laguna,38.4210,-121.4238,34.69,40,0,3.60,US,1694741394
48,48,lefka,35.1106,32.8506,23.08,41,0,0.88,CY,1694741695
57,57,al bawiti,28.3492,28.8659,20.81,72,0,3.21,EG,1694741699


### Step 3: Create a new DataFrame called `HotelsDF`.

In [18]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity

HotelsDF = IdealWeatherDF[["City","Country","Lat","Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API

HotelsDF["Hotel Name"] = ""

# Display the data

HotelsDF.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
11,mengyin,CN,35.7069,117.9264,47,
24,quincy,US,42.2529,-71.0023,78,
29,laguna,US,38.4210,-121.4238,40,
48,lefka,CY,35.1106,32.8506,41,
57,al bawiti,EG,28.3492,28.8659,72,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [19]:
# Set parameters to search for a hotel

radius = 10000
params = {
    "categories": "accomodation.hotel",
    "apiKey": geoapify_key,
    "limit": 20
}

# Print a message to follow up the hotel search

print("Starting hotel search")

# Iterate through HotelsDF DataFrame
for index, row in HotelsDF.iterrows():

    # get latitude, longitude from the DataFrame

    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary

    params["filter"] = f"circle:{longitude}, {latitude}, {radius}"
    params["bias"] = f"proximity:{longitude}, {latitude}"
    
    # Set base URL

    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionary

    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format

    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the HotelsDF DataFrame

    try:
        HotelsDF.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):

        # If no hotel is found, set the hotel name as "No hotel found".
        
        HotelsDF.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{HotelsDF.loc[index, 'City']} - nearest hotel: {HotelsDF.loc[index, 'Hotel Name']}")

# Display sample data
HotelsDF

Starting hotel search
mengyin - nearest hotel: No hotel found
quincy - nearest hotel: No hotel found
laguna - nearest hotel: No hotel found
lefka - nearest hotel: No hotel found
al bawiti - nearest hotel: No hotel found
la passe - nearest hotel: No hotel found
chifeng - nearest hotel: No hotel found
luena - nearest hotel: No hotel found
bethel - nearest hotel: No hotel found
hamilton - nearest hotel: No hotel found
kasane - nearest hotel: No hotel found
tsiombe - nearest hotel: No hotel found
yarmouth - nearest hotel: No hotel found
alice springs - nearest hotel: No hotel found
waddan - nearest hotel: No hotel found
newman - nearest hotel: No hotel found
morinville - nearest hotel: No hotel found
richmond - nearest hotel: No hotel found
ojinaga - nearest hotel: No hotel found
alvaraes - nearest hotel: No hotel found
camalu - nearest hotel: No hotel found
umluj - nearest hotel: No hotel found
awjilah - nearest hotel: No hotel found
hotan - nearest hotel: No hotel found
saint-pierre - ne

,City,Country,Lat,Lng,Humidity,Hotel Name
11,mengyin,CN,35.7069,117.9264,47,No hotel found
24,quincy,US,42.2529,-71.0023,78,No hotel found
29,laguna,US,38.4210,-121.4238,40,No hotel found
48,lefka,CY,35.1106,32.8506,41,No hotel found
57,al bawiti,EG,28.3492,28.8659,72,No hotel found
...,...,...,...,...,...,...
563,remire-montjoly,GF,4.9167,-52.2667,83,No hotel found
568,mccook,US,40.2019,-100.6257,65,No hotel found
569,griffith,US,41.5284,-87.4237,75,No hotel found
570,mugia,ES,43.1041,-9.2179,79,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [20]:
%%capture --no-display

# Configure map plot

MapPlot2 = HotelsDF.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = .5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    hover_cols = ["Hotel Name", "Country"]
)

MapPlot2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)